In [8]:
# Set-up
import numpy as np
import scipy.linalg as sp

In [9]:
%%capture
%run forward-elimination.ipynb
%run backward-substitution.ipynb
%run LU-factorization.ipynb

### Partial pivoting with LU factorization
#### Row swapper

In [21]:
def row_swap(A,P):
    if len(P) < 1:
        # Check if non-empty
        print(1)
        return None
    if len(P) != len(P[0]):
        # Check if square
        print(2)
        return None
    if len(P) != len(A):
        print(3)
        return None
    # Check if only 1s and 0s
    for row in P:
        for num in row:
            if num != 1 and num != 0:
                return None
    # Check if not permutated I
    for j in range(len(P)):
        found_one = False
        for i in range(len(P[0])):
            #if P[i,j] == 1 && 
            pass
            
    swaps = []
    for i in range(len(P)):
        for j in range(len(P)):
            if P[j,i] == 1:
                swaps.append((i,j))
    A_swapped = np.copy(A)
    for swap in swaps:
        A_swapped[swap[1]] = np.copy(A[swap[0]])
    return A_swapped

In [23]:
P = np.array([[0,1,0],[0,0,1],[1,0,0]])
print(P)

A1 = np.array([[1,2,2],[4,4,2],[4,6,4]])
print(A1)
print("################")
A1 = row_swap(A1,P)
print(A1)

[[0 1 0]
 [0 0 1]
 [1 0 0]]
[[1 2 2]
 [4 4 2]
 [4 6 4]]
################
[[4 4 2]
 [4 6 4]
 [1 2 2]]


#### Pivoting forward elimination

In [24]:
def forward_elim_pivot(A,b):
    A_copy = np.copy(A)
    b_copy = np.copy(b)
    M = np.identity(len(A))
    Ms = [] # This stores the elimination matrices in order [M_1, M_2, M_3], useful for L.
    Ps = []
    for k in range(len(A[0])):
        # This parts do the partial pivoting
        P = np.identity(len(A))
        max_idx = k
        for j in range(k+1,len(A)):
            if A_copy[k,j] > A_copy[k,max_idx]:
                max_idx = j
        if max_idx != k:
            P[[k, max_idx]] = P[[max_idx, k]]
            A_copy = row_swap(A_copy, P)
        Ps.append(P)
        # This part creates the elimination matrix      
        M_k = np.copy(M)
        pivot = A_copy[k,k]
        for i in range(k+1, len(A)):
            M_k[i, k] = -(A_copy[i,k]/pivot)
        Ms.append(M_k)
        A_copy = M_k @ A_copy
        b_copy = M_k @ b_copy
    return A_copy, b_copy, Ms, Ps

#### Pivoting LU factorization

In [25]:
def LU_pivot(A,b):
    U, z, Ms, Ps = forward_elim_pivot(A,b)
    L = np.identity(len(A))
    P = Ps[-1]
    for i in range(len(Ps)-1,-1,-1):
        P = P @ Ps[i]
    for i in range(len(Ms)):
        L = L @ Ps[i] @ np.linalg.inv(Ms[i])
    L = row_swap(L,P)
    
    return L, U, b, P

In [26]:
A1 = np.array([[1,2,2],[4,4,2],[4,6,4]])
b1 = np.array([[3],[6],[10]])

print("############### ROW SWAPS ###############")
A1, b1, Ms, Ps = forward_elim_pivot(A1,b1)
for P in Ps:
    print(P)
print("#########################################\n")

A2 = np.array([[1,2,2],[4,4,2],[4,6,4]])
b2 = np.array([[3],[6],[10]])
print("########### PARTIAL PIVOTING ############")
L, U, b, P = LU_pivot(A2,b2)
print("P: \n",P)
print("L: \n",L)
print("U: \n",U)
print("#########################################\n")

############### ROW SWAPS ###############
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
#########################################

########### PARTIAL PIVOTING ############
P: 
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
L: 
 [[1.   0.   0.  ]
 [1.   1.   0.  ]
 [0.25 0.5  1.  ]]
U: 
 [[4.  4.  2. ]
 [0.  2.  2. ]
 [0.  0.  0.5]]
#########################################

